# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
from glob import glob

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
raw_fold = 'D:/data/big2/BD1/raw'
ods_fold = 'D:/data/big2/ODS'
sample_fold = 'D:/data/big2/BD1/sample'
analysis_fold = 'D:/data/big2/BD1/an'

# Load

## 4등급 경유 result&정기&정밀(최신검사)

In [4]:
# about 20.4s
name = 'result(4등급_경유)&정기&정밀(최신검사)(2023.03.22)'
file_name = f'{name}.csv'
result = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148315 entries, 0 to 1148314
Data columns (total 59 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   차대번호       1148315 non-null  object 
 1   차량번호       1148315 non-null  object 
 2   제원관리번호     1148315 non-null  object 
 3   차종         1148315 non-null  object 
 4   용도         1148315 non-null  object 
 5   최초등록일      1148315 non-null  int64  
 6   연식         1148315 non-null  int64  
 7   제작일자       1148313 non-null  float64
 8   검사유효일      1148312 non-null  float64
 9   배출가스인증번호   1141202 non-null  object 
 10  등급         1148315 non-null  int64  
 11  차명         1148315 non-null  object 
 12  차종분류       1148315 non-null  object 
 13  차종유형       1148315 non-null  object 
 14  자동차형식      1148314 non-null  object 
 15  제작사명       1148297 non-null  object 
 16  연료         1148315 non-null  object 
 17  엔진형식       1148315 non-null  object 
 18  총중량        1148315 non-null  int64  
 19  

In [5]:
result.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정'],
      dtype='object')

In [6]:
result['등급_수정'].value_counts(dropna=False)

4    1149291
3       4396
5        126
Name: 등급_수정, dtype: int64

In [8]:
result['연료'].value_counts(dropna=False)

경유    1153813
Name: 연료, dtype: int64

In [9]:
result['말소여부'].value_counts(dropna=False)

N    1153813
Name: 말소여부, dtype: int64

In [10]:
result[['배인번호_수정', '등급_수정', 'DPF유무_수정']].isnull().sum()

배인번호_수정     0
등급_수정       0
DPF유무_수정    0
dtype: int64

In [11]:
result1 = result.loc[result['등급_수정'] == 4].reset_index(drop=True)
result1.shape

(1149291, 31)

## 정기&정밀검사 최신

In [12]:
# about 4m 21s
# modin about : 49s
name = '[ODS]정기&정밀검사(최신검사만)(2022.12.28)'
file_name = f'{name}.csv'
tin = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
tin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22044208 entries, 0 to 22044207
Data columns (total 42 columns):
 #   Column     Dtype  
---  ------     -----  
 0   사용연료       object 
 1   주행거리       object 
 2   차명         object 
 3   차량번호       object 
 4   차대번호       object 
 5   차종         object 
 6   차량연식       int64  
 7   엔진형식       object 
 8   검사방법       object 
 9   검사일자       int64  
 10  검사종류       object 
 11  검사판정시각     object 
 12  검사판정       object 
 13  산소값        float64
 14  이산화탄소값     float64
 15  무부하매연판정1   object 
 16  무부하매연판정2   object 
 17  무부하매연판정3   object 
 18  무부하매연판정4   object 
 19  무부하매연판정5   object 
 20  무부하매연판정6   object 
 21  무부하매연허용치1  float64
 22  무부하매연허용치2  float64
 23  무부하매연허용치3  float64
 24  무부하매연허용치4  float64
 25  무부하매연허용치5  float64
 26  무부하매연허용치6  float64
 27  무부하매연측정치1  float64
 28  무부하매연측정치2  float64
 29  무부하매연측정치3  float64
 30  무부하매연측정치4  float64
 31  무부하매연측정치5  float64
 32  무부하매연측정치6  float64
 33  차량용도       object 
 34  최대출력허용치    float

# 전처리

### 8MY-PA-14-3 배인번호 수정
- 8MY-PA-14-3 -> 8MY-PA-14-03

In [13]:
result.loc[result['배인번호_수정'] == '8MY-PA-14-3'].shape

(412, 31)

In [14]:
result1 = result.copy()

In [15]:
result1.loc[result1['배인번호_수정'] == '8MY-PA-14-3', '배인번호_수정'] = '8MY-PA-14-03'
result1.shape

(1153813, 31)

### 9MY-AD-14-04 등급 수정
- 9MY-AD-14-04 : 등급_수정 오류, 현재 4등급 -> 3등급

In [16]:
result1.loc[result1['배인번호_수정'] == '9MY-AD-14-04', '등급_수정'].unique()

array([4], dtype=int64)

In [17]:
result1.loc[result1['배인번호_수정'] == '9MY-AD-14-04'].shape

(976, 31)

In [18]:
result2 = result1.copy()

In [19]:
result2.loc[result2['배인번호_수정'] == '9MY-AD-14-04', '등급_수정'] = 3

In [20]:
result2.loc[result2['배인번호_수정'] == '9MY-AD-14-04', '등급_수정'].unique()

array([3], dtype=int64)

### 3등급 차량 중 이상한 차량 샘플

In [21]:
result2.loc[(result2['등급_수정'] == 3) & (result2['배인번호_수정'] == '확인불가')].shape

(3, 31)

In [22]:
result2.loc[(result2['등급_수정'] == 3) & (result2['배인번호_수정'] == '확인불가'), ['배인번호_수정', '차명', '제작사명', '자동차형식', '엔진형식']]

,배인번호_수정,차명,제작사명,자동차형식,엔진형식
1147742,확인불가,뉴파워트럭,현대자동차(주),HD8CT-20ESHa-T6,D6HC
1147743,확인불가,뉴파워트럭,현대자동차(주),HD8CT-20ESHa-T6,D6HC
1147744,확인불가,뉴파워트럭,현대자동차(주),HD8CT-20ESHa-T6,D6HC


#### 샘플 출력

In [23]:
# result2.loc[(result2['등급_수정'] == 3) & (result2['배인번호_수정'] == '확인불가')].to_excel(os.path.join(raw_fold, '샘플_3등급_확인불가_차량.xlsx'), index=False)

### [출력] result2

In [24]:
# today_date = datetime.today().strftime("%Y.%m.%d")
# today_date

In [25]:
# # about 15s
# result2.to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종({today_date}).csv'), index=False, encoding='cp949')

## 등급_수정 4등급만 추출

In [26]:
result2['등급_수정'].value_counts(dropna=False)

4    1148315
3       5372
5        126
Name: 등급_수정, dtype: int64

In [27]:
result3 = result2.loc[result2['등급_수정'] == 4]
result3.shape

(1148315, 31)

## 정기&정밀 검사 특정 기간 데이터 추출

In [28]:
tin['검사일자'].dtype

dtype('int64')

In [29]:
tin1 = tin.loc[(tin['검사일자'] >= 20200108) & (tin['검사일자'] <= 20230107)]
tin1.shape

(20714502, 42)

In [30]:
tin1['검사일자'].describe()

count   20714502.00
mean    20215015.93
std         6694.21
min     20200108.00
25%     20210614.00
50%     20220117.00
75%     20220623.00
max     20230107.00
Name: 검사일자, dtype: float64

## result와 정기&정밀 검사 파일 병합

In [31]:
# about 45s
# modin about error
df = result3.merge(tin1, on='차대번호', how='left')
df.shape

(1148315, 72)

In [32]:
df.columns

Index(['차대번호', '차량번호_x', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자',
       '검사유효일', '배출가스인증번호', '등급', '차명_x', '차종분류', '차종유형', '자동차형식', '제작사명',
       '연료', '엔진형식_x', '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명',
       '소유자구분', '말소여부', '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '사용연료', '주행거리',
       '차명_y', '차량번호_y', '차종', '차량연식', '엔진형식_y', '검사방법', '검사일자', '검사종류',
       '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1', '무부하매연판정2', '무부하매연판정3',
       '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1', '무부하매연허용치2',
       '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6', '무부하매연측정치1',
       '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6', '차량용도',
       '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간', '정기정밀', '검사 검사소코드',
       '검사유효기간'],
      dtype='object')

In [33]:
df = df[['차대번호', '차량번호_x', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일', '배출가스인증번호', '등급', '차명_x', '차종분류', '차종유형', '자동차형식', '제작사명',
         '연료', '엔진형식_x', '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부', '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 
         '주행거리', '차량연식', '검사방법', '검사일자', '검사종류', '검사판정시각', '검사판정', 
         '산소값', '이산화탄소값', 
         '무부하매연판정1', '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', 
         '무부하매연허용치1', '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6', 
         '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5', '무부하매연측정치6', 
         '정기정밀']]
df = df.rename(columns={'차량번호_x':'차량번호', '차명_x':'차명', '엔진형식_x':'엔진형식', '차종_원본':'차종'})
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '주행거리', '차량연식', '검사방법', '검사일자',
       '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1', '무부하매연판정2',
       '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '정기정밀'],
      dtype='object')

In [34]:
df.shape

(1148315, 59)

In [35]:
df['검사일자'].describe()

count    1120201.00
mean    20217159.00
std         5579.48
min     20200108.00
25%     20211101.00
50%     20220330.00
75%     20220730.00
max     20230107.00
Name: 검사일자, dtype: float64

### [출력] result와 정기&정밀(최신검사)

In [36]:
# # about 28s
# df.to_csv(os.path.join(raw_fold, 'result(4등급_경유)&정기&정밀(최신검사)(2023.03.22).csv'), index=False, encoding='cp949')

## str 타입으로 변경
- 배인번호_수정
- 엔진형식
- 검사판정

In [223]:
df[['제작사명', '배인번호_수정', '자동차형식', '엔진형식', '연식', '검사판정', '검사방법']].isnull().sum()

제작사명           0
배인번호_수정        0
자동차형식          0
엔진형식           0
연식             0
검사판정           0
검사방법       28114
dtype: int64

In [224]:
df['제작사명'] = df['제작사명'].astype('str')
df['배인번호_수정'] = df['배인번호_수정'].astype('str')
df['자동차형식'] = df['자동차형식'].astype('str')
df['엔진형식'] = df['엔진형식'].astype('str')
df['검사판정'] = df['검사판정'].astype('str')
df['검사방법'] = df['검사방법'].astype('str')

In [225]:
df[['제작사명', '배인번호_수정', '자동차형식', '엔진형식', '검사판정', '검사방법']].isnull().sum()

제작사명       0
배인번호_수정    0
자동차형식      0
엔진형식       0
검사판정       0
검사방법       0
dtype: int64

## 차대번호 10자리 연식

In [226]:
df1 = df.copy()

In [227]:
df1['vin10'] = df1['차대번호'].str[9]

In [228]:
vin10_dict = {'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017}

In [229]:
df1['vin10_year'] = df1['vin10'].map(vin10_dict, na_action='ignore')

## check 파일 정리

In [230]:
check.head()

,제작사,배출가스 인증번호,차명,Result1,Result2,검증 후 차량 대수,DPF장착여부,배출가스 등급
0,현대자동차(주),3MY-HD-14,포터Ⅱ(PORTERⅡ),0.00,8.00,8,N,5.00
1,현대자동차(주),3MY-HD-15,포터Ⅱ(PORTERⅡ),0.00,1.00,1,N,5.00
2,아우디폭스바겐코리아(주),4MY-AD-20,아우디 A6 3.0 TDI Quattro,29.00,1.00,30,N,4.00
3,자일대우상용차 주식회사,4MY-DB-03,BX212,9.00,0.00,9,N,4.00
4,자일대우상용차 주식회사,4MY-DB-04,BS106,24.00,0.00,24,N,4.00


In [231]:
check.tail()

,제작사,배출가스 인증번호,차명,Result1,Result2,검증 후 차량 대수,DPF장착여부,배출가스 등급
169,볼보트럭코리아(주),FMY-VTK-14-4,FH 트랙터,0.00,49.00,49,Y,3.00
170,미기입,확인안됨,코란도KJ,0.00,1.00,1,N,5.00
171,미기입,확인안됨,기아뉴복사17,0.00,1.00,1,N,5.00
172,미기입,확인안됨,NaN,3.00,6656.00,6659,알수없음,4.00
173,합계,NaN,NaN,NaN,NaN,1153813,NaN,NaN


In [232]:
check1 = check.iloc[:170]
check1.tail()

,제작사,배출가스 인증번호,차명,Result1,Result2,검증 후 차량 대수,DPF장착여부,배출가스 등급
165,자일대우상용차 주식회사,FMY-DB-14-54,BH090,13.00,0.00,13,Y,3.00
166,볼보트럭코리아(주),FMY-VTK-14-1,FH 트랙터,0.00,1780.00,1780,Y,3.00
167,볼보트럭코리아(주),FMY-VTK-14-2,FM 카고,0.00,87.00,87,Y,3.00
168,볼보트럭코리아(주),FMY-VTK-14-3,FL 카고,0.00,76.00,76,Y,3.00
169,볼보트럭코리아(주),FMY-VTK-14-4,FH 트랙터,0.00,49.00,49,Y,3.00


In [233]:
check1.shape

(170, 8)

In [234]:
emis_list = check1['배출가스 인증번호'].unique()
len(emis_list)

170

## 배인번호 교집합 확인

In [235]:
# 해당 배인번호가 아닌 샘플
len(df1['배출가스인증번호'].unique()), len(df1['배인번호_수정'].unique()), len(set(emis_list))

(190, 140, 170)

In [236]:
set(df1['배인번호_수정'].unique()) - set(emis_list)

{'확인불가'}

## 배인번호 17자리 샘플

In [237]:
df2y = df1.loc[df1['차대번호'].str.len() == 17].reset_index(drop=True)
df2n = df1.loc[df1['차대번호'].str.len() != 17].reset_index(drop=True)
df2y.shape, df2n.shape

((1148306, 61), (9, 61))

### [출력] 샘플-배인번호17자리 아닌

In [239]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
df2n.to_excel(os.path.join(sample_fold, f'통합_샘플_차대번호17자리아닌_{today_date}.xlsx'), index=False)

## 배인 연식과 연식 동일한 샘플

In [240]:
df3y = df2y.loc[df2y['vin10_year'] == df2y['연식']].reset_index(drop=True)
df3n = df2y.loc[df2y['vin10_year'] != df2y['연식']].reset_index(drop=True)
df3y.shape, df3n.shape

((1148246, 61), (60, 61))

### [출력] 샘플-배인연식 다른

In [241]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
df3n.to_excel(os.path.join(sample_fold, f'통합_샘플_차대번호17자리_차량연식다른_{today_date}.xlsx'), index=False)

# [test] 배출가스 인증번호별 분석
- [해결] issue
    - 배인번호 없음 : 9MY-KM-14-22
        - 차명 : 쏘렌토
        - 등급변경 : 3 -> 4

## [로드] 배인번호 파일

In [242]:
# 5MY-BK-14-07
# 6MY-BK-13-08
# 6MY-MK-14-02

# 5MY-HD-24-35
# 8MY-PA-14-03

In [243]:
test_df = pd.DataFrame()

In [244]:
emis_no = '5MY-HD-24-35'
gas_df = df1.loc[df1['배인번호_수정'] == emis_no]
gas_df.shape

(1089, 61)

### 전처리
- 제거
    - 차대번호 17자리 아닌 샘플
    - 차대번호 10자리와 차량연식이 다른 샘플

#### [출력] 차대번호17자리 아닌 샘플

In [245]:
vin17y = gas_df[(gas_df['차대번호'].str.len() == 17)].reset_index(drop=True)
vin17n = gas_df[(gas_df['차대번호'].str.len() != 17)].reset_index(drop=True)
vin17y.shape, vin17n.shape

((1089, 61), (0, 61))

In [246]:
# vin17n.to_excel(os.path.join(sample_fold, f'샘플_차대번호17자리아닌_{emis_no}.xlsx'), index=False)

#### [출력] 차대번호10자리 연도와 차량연식 다른 샘플

In [247]:
vin17y_vin10y = vin17y[vin17y['vin10_year'] == vin17y['연식']].reset_index(drop=True)
vin17y_vin10n = vin17y[vin17y['vin10_year'] != vin17y['연식']].reset_index(drop=True)
vin17y_vin10y.shape, vin17y_vin10n.shape

((1089, 61), (0, 61))

In [248]:
# vin17y_vin10n.to_excel(os.path.join(sample_fold, f'샘플_차대번호17자리_차량연식다른_{emis_no}.xlsx'), index=False)

### 분석
- 제원관리번호별, 자동차형식별, 엔진형식별, 검사판정별 무부하매연측정치1 통계

In [249]:
vin17y_vin10y['무부하매연측정치1'].dtype

dtype('float64')

In [250]:
vin17y_vin10y['무부하매연측정치1'].isnull().sum()

56

In [251]:
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

In [252]:
vin17y_vin10y.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '주행거리', '차량연식', '검사방법', '검사일자',
       '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1', '무부하매연판정2',
       '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '정기정밀', 'vin10', 'vin10_year'],
      dtype='object')

In [253]:
vin17y_vin10y['검사방법'].unique()

array(['부하검사(KD-147)', '무부하검사(급가속)', 'nan'], dtype=object)

In [254]:
vin17y_vin10y['검사종류'].unique()

array(['2부재검사(종합)', '종합검사(경과)', '종합검사', '정기검사', '1부재검사(종합)', nan,
       '재검사(2부재검사)', '재검사(1부재검사)', '정기(경과)검사'], dtype=object)

In [255]:
g = vin17y_vin10y.groupby(['제작사명', '배인번호_수정', '제원관리번호', '자동차형식', '엔진형식', '검사방법', '검사판정'], dropna=False).agg({'차대번호':'count', '무부하매연측정치1':['mean', 'min', 'max']}).pipe(flat_cols).round(2).reset_index()
g = g.rename(columns={'배인번호_수정':'배출가스인증번호', '차대번호/count':'대수', '무부하매연측정치1/mean':'mean', '무부하매연측정치1/min':'min', '무부하매연측정치1/max':'max'})

In [256]:
g

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사방법,검사판정,대수,mean,min,max
0,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,nan,nan,20,NaN,NaN,NaN
1,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),N,16,32.88,6.00,66.00
2,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),Y,93,12.45,0.00,39.00
3,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),N,41,48.08,26.00,89.00
4,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),Y,335,11.80,0.00,25.00
5,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,nan,nan,5,NaN,NaN,NaN
6,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),N,3,15.33,0.00,36.00
7,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),Y,5,8.80,0.00,21.00
8,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,부하검사(KD-147),Y,24,12.33,0.00,24.00
9,현대자동차(주),5MY-HD-24-35,A0810006700381105,MC-D15-A1SA,D4FA,부하검사(KD-147),Y,1,9.00,9.00,9.00


In [257]:
g.groupby(['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사방법'])['대수'].transform('sum')

0      20
1     109
2     109
3     376
4     376
5       5
6       8
7       8
8      24
9       1
10     14
11     70
12     70
13    271
14    271
15      1
16     26
17     26
18      3
19      9
20      1
21      2
22      4
23      4
24      3
25     21
26     21
27     81
28     81
29      4
30      4
31      1
32      1
33      3
34     27
35     27
Name: 대수, dtype: int64

In [258]:
g['검사판정비율'] = g['대수'] / g.groupby(['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사방법'])['대수'].transform('sum')
g

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사방법,검사판정,대수,mean,min,max,검사판정비율
0,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,nan,nan,20,NaN,NaN,NaN,1.00
1,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),N,16,32.88,6.00,66.00,0.15
2,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),Y,93,12.45,0.00,39.00,0.85
3,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),N,41,48.08,26.00,89.00,0.11
4,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),Y,335,11.80,0.00,25.00,0.89
5,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,nan,nan,5,NaN,NaN,NaN,1.00
6,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),N,3,15.33,0.00,36.00,0.38
7,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),Y,5,8.80,0.00,21.00,0.62
8,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,부하검사(KD-147),Y,24,12.33,0.00,24.00,1.00
9,현대자동차(주),5MY-HD-24-35,A0810006700381105,MC-D15-A1SA,D4FA,부하검사(KD-147),Y,1,9.00,9.00,9.00,1.00


In [259]:
test_df = pd.concat([test_df, g], ignore_index=True)

In [260]:
test_df

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사방법,검사판정,대수,mean,min,max,검사판정비율
0,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,nan,nan,20,NaN,NaN,NaN,1.00
1,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),N,16,32.88,6.00,66.00,0.15
2,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),Y,93,12.45,0.00,39.00,0.85
3,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),N,41,48.08,26.00,89.00,0.11
4,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),Y,335,11.80,0.00,25.00,0.89
5,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,nan,nan,5,NaN,NaN,NaN,1.00
6,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),N,3,15.33,0.00,36.00,0.38
7,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),Y,5,8.80,0.00,21.00,0.62
8,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,부하검사(KD-147),Y,24,12.33,0.00,24.00,1.00
9,현대자동차(주),5MY-HD-24-35,A0810006700381105,MC-D15-A1SA,D4FA,부하검사(KD-147),Y,1,9.00,9.00,9.00,1.00


In [160]:
# # 방법1
# concat_two = pd.DataFrame()
# for two in g['제원관리번호'].unique():
#     for three in g['엔진형식'].unique():
#         for four in g['검사방법'].unique():
#             temp = g[(g['제원관리번호'] == two) & (g['엔진형식'] == three) & (g['검사방법'] == four)].copy()
#             temp['sum'] = temp['대수'].sum()
#             temp['검사판정비율'] = temp['대수'] / temp['sum']
#             concat_two = pd.concat([concat_two, temp], ignore_index=True)

In [161]:
# concat_two

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사방법,검사판정,대수,mean,min,max,sum,검사판정비율
0,(주)볼보자동차코리아,8MY-PA-14-03,920003000001308,B71,D5244T,무부하검사(급가속),N,1,0.00,0.00,0.00,50,0.02
1,(주)볼보자동차코리아,8MY-PA-14-03,920003000001308,B71,D5244T,무부하검사(급가속),Y,49,1.73,0.00,21.00,50,0.98
2,(주)볼보자동차코리아,8MY-PA-14-03,920003000011308,B71,D5244T,무부하검사(급가속),N,1,0.00,0.00,0.00,137,0.01
3,(주)볼보자동차코리아,8MY-PA-14-03,920003000011308,B71,D5244T,무부하검사(급가속),Y,136,0.66,0.00,25.00,137,0.99
4,(주)볼보자동차코리아,8MY-PA-14-03,920003000021309,B70,D5244T,무부하검사(급가속),Y,86,0.34,0.00,6.00,86,1.00
5,(주)볼보자동차코리아,8MY-PA-14-03,920003000031310,B70,D5244T,무부하검사(급가속),N,1,0.00,0.00,0.00,185,0.01
6,(주)볼보자동차코리아,8MY-PA-14-03,920003000031310,B70,D5244T,무부하검사(급가속),Y,184,0.21,0.00,15.00,185,0.99


In [162]:
# test_df = pd.concat([test_df, concat_two], ignore_index=True)

In [163]:
# test_df

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사방법,검사판정,대수,mean,min,max,sum,검사판정비율
0,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),N,16,32.88,6.00,66.00,109,0.15
1,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,무부하검사(급가속),Y,93,12.45,0.00,39.00,109,0.85
2,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),N,41,48.08,26.00,89.00,376,0.11
3,현대자동차(주),5MY-HD-24-35,A0810006700341105,MC-D15-A1A,D4FA,부하검사(KD-147),Y,335,11.80,0.00,25.00,376,0.89
4,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),N,3,15.33,0.00,36.00,8,0.38
5,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,무부하검사(급가속),Y,5,8.80,0.00,21.00,8,0.62
6,현대자동차(주),5MY-HD-24-35,A0810006700351105,MC-D15-M1A,D4FA,부하검사(KD-147),Y,24,12.33,0.00,24.00,24,1.00
7,현대자동차(주),5MY-HD-24-35,A0810006700381105,MC-D15-A1SA,D4FA,부하검사(KD-147),Y,1,9.00,9.00,9.00,1,1.00
8,현대자동차(주),5MY-HD-24-35,A0810006700421105,MC-D15GS-A1A,D4FA,무부하검사(급가속),N,9,37.22,8.00,96.00,70,0.13
9,현대자동차(주),5MY-HD-24-35,A0810006700421105,MC-D15GS-A1A,D4FA,무부하검사(급가속),Y,61,12.79,0.00,41.00,70,0.87


In [74]:
# g.to_excel(os.path.join(analysis_fold, f'통계_제번별_차형식별_엔진형식별_검사판정별_무부하매연측정치1_{emis_no}.xlsx'))

In [76]:
# gas_nm_list = check_df['배출가스인증번호'].unique()
# len(gas_nm_list)

In [77]:
check1.columns

Index(['제작사', '배출가스 인증번호', '차명', 'Result1', 'Result2', '검증 후 차량 대수', 'DPF장착여부',
       '배출가스 등급'],
      dtype='object')

In [78]:
emis_list = check1['배출가스 인증번호'].unique()
len(emis_list)

170

In [79]:
emis_list[:5]

array(['3MY-HD-14', '3MY-HD-15', '4MY-AD-20', '4MY-DB-03', '4MY-DB-04'],
      dtype=object)

In [80]:
# productor_dict = {
#     'HD':'현대자동차(주)', 
#     'AD':'아우디폭스바겐코리아(주)', 
#     'PP':'아우디폭스바겐코리아(주)', 
#     'DB':'자일대우상용차 주식회사',
#     'DK':'다임러트럭코리아(주)', 
#     'DK':'에프씨에이코리아(주)', 
#     'KM':'기아자동차(주)', 
#     'PA':'(주)볼보자동차코리아', 
#     'BK':'비엠더블유코리아(주)',
#     'HB':'한불모터스(주)', 
#     'SK':'스카니아코리아그룹(주)', 
#     'SY':'쌍용자동차(주)', 
#     'VTK':'볼보트럭코리아(주)', 
#     'DC':'타타대우상용차(주)', 
#     'DW':'한국지엠주식회사', 
#     'GK':'지엠아시아퍼시픽지역본부 주식회사', 
#     'MK':'메르세데스벤츠코리아(주)', 
#     'MT':'만트럭버스코리아(주)', 
#     'SM':'르노삼성자동차(주)', 
#     'FK':'포드세일즈서비스코리아(유)', 
#     'JL':'(주)재규어랜드로버코리아', 
#     }

# 배인번호별 분석2

In [75]:
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

In [164]:
# about 24m 51s
total_g_df = pd.DataFrame()

for one in tqdm(df3y['배인번호_수정'].unique()):
    # 배인번호별 df
    gas_df = df3y.loc[df3y['배인번호_수정'] == str(one)].reset_index(drop=True)

    if gas_df.shape[0] != 0:
        # 제번별, 차형식별, 엔진형식별, 검사판정별 무부하매연측정치1 통계
        g = gas_df.groupby(['제작사명', '배인번호_수정', '제원관리번호', '자동차형식', '엔진형식', '검사방법', '검사판정'], dropna=False).agg({'차대번호':'count', '무부하매연측정치1':['mean', 'min', 'max']}).pipe(flat_cols).round(2).reset_index()
        g = g.rename(columns={'배인번호_수정':'배출가스인증번호', '차대번호/count':'대수', '무부하매연측정치1/mean':'mean', '무부하매연측정치1/min':'min', '무부하매연측정치1/max':'max'})
        # 하나의 배인번호에서 제번별 엔진형식별 비율 계산
        concat_two = pd.DataFrame()
        for two in g['제원관리번호'].unique():
            for three in g['엔진형식'].unique():
                for four in g['검사방법'].unique():
                    temp = g[(g['제원관리번호'] == two) & (g['엔진형식'] == three) & (g['검사방법'] == four)].copy()
                    temp['sum'] = temp['대수'].sum()
                    temp['검사판정비율'] = temp['대수'] / temp['sum']
                    concat_two = pd.concat([concat_two, temp], ignore_index=True)
        # 종합 - 통계
        total_g_df = pd.concat([total_g_df, concat_two], ignore_index=True)
    else:
        print(f'오류 배인번호 : {one}')
        pass

100%|██████████| 140/140 [24:51<00:00, 10.65s/it]  


# 배인번호별 분석3
- 기존 '분석2' 알고리즘보다 비약적인 속도 향상
    - 24분 51초 -> 약 12초(11.9초)
        - 시간 99% 절감, 처리 속도 124배 향상

In [261]:
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

In [262]:
# about 11.9s
# 최적화 24m 51s -> 11.9s 시간 99% 절감, 처리 속도 124배 증가
total_g_df = pd.DataFrame()

for one in tqdm(df3y['배인번호_수정'].unique()):
    # 배인번호별 df
    gas_df = df3y.loc[df3y['배인번호_수정'] == str(one)].reset_index(drop=True)

    if gas_df.shape[0] != 0:
        # 제번별, 차형식별, 엔진형식별, 검사판정별 무부하매연측정치1 통계
        g = gas_df.groupby(['제작사명', '배인번호_수정', '제원관리번호', '자동차형식', '엔진형식', '검사방법', '검사판정'], dropna=False).agg({'차대번호':'count', '무부하매연측정치1':['mean', 'min', 'max']}).pipe(flat_cols).round(2).reset_index()
        g = g.rename(columns={'배인번호_수정':'배출가스인증번호', '차대번호/count':'대수', '무부하매연측정치1/mean':'mean', '무부하매연측정치1/min':'min', '무부하매연측정치1/max':'max'})
        # 하나의 배인번호에서 제번별 엔진형식별 비율 계산
        g['검사판정비율'] = g['대수'] / g.groupby(['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사방법'])['대수'].transform('sum')
        # 종합 - 통계
        total_g_df = pd.concat([total_g_df, g], ignore_index=True)
    else:
        print(f'오류 배인번호 : {one}')
        pass

100%|██████████| 140/140 [00:11<00:00, 11.71it/s]


In [263]:
total_g_df.head()

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사방법,검사판정,대수,mean,min,max,검사판정비율
0,아우디폭스바겐코리아(주),4MY-AD-20,2420000100031300,4F,BMK,nan,nan,3,NaN,NaN,NaN,1.00
1,아우디폭스바겐코리아(주),4MY-AD-20,2420000100031300,4F,BMK,무부하검사(급가속),N,2,36.00,21.00,51.00,0.08
2,아우디폭스바겐코리아(주),4MY-AD-20,2420000100031300,4F,BMK,무부하검사(급가속),Y,24,13.42,0.00,31.00,0.92
3,자기인증면제차량,4MY-AD-20,31124022261309,4F,BMK,무부하검사(급가속),Y,1,2.00,2.00,2.00,1.00
4,(주)볼보자동차코리아,4MY-PA-06,920000300131205,2.0LN5DR,204D,nan,nan,1,NaN,NaN,NaN,1.00


In [264]:
total_g_df.shape

(15724, 12)

In [265]:
total_g_df1 = total_g_df.loc[total_g_df['배출가스인증번호'] != '확인불가']
total_g_df1.shape

(13217, 12)

## [출력] 통계 배인번호별 매연값

In [266]:
# about 2s
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
total_g_df1.to_excel(os.path.join(analysis_fold, f'통계_배인번호별_매연_{today_date}.xlsx'), index=False)

In [267]:
df3y.shape

(1148246, 61)

In [268]:
total_g_df1['대수'].sum()

1141598

In [269]:
df3y[df3y['배인번호_수정'] == '확인불가'].shape

(6648, 61)

In [270]:
df2n[df2n['배인번호_수정'] == '확인불가'].shape

(0, 61)

In [271]:
df3n[df3n['배인번호_수정'] == '확인불가'].shape

(8, 61)

In [272]:
df[df['배인번호_수정'] == '확인불가'].shape

(6656, 59)

## result 파일 통계

In [221]:
result2.loc[result2['배인번호_수정'] == '8MY-PA-14-03'].shape

(462, 31)

In [222]:
result2.groupby(['등급_수정', '배인번호_수정'])['차대번호'].count()

등급_수정  배인번호_수정     
3      5MY-AD-13-07    118
       5MY-AD-13-13     51
       6MY-AD-13-11    493
       7MY-AD-13-01    565
       7MY-AD-13-05     26
                      ... 
5      5MY-KM-24-32      6
       5MY-SY-24-48      2
       6MY-DC-23-17      7
       6MY-HD-23-55      3
       확인불가              2
Name: 차대번호, Length: 172, dtype: int64

### [출력]

In [98]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
result2.groupby(['등급_수정', '배인번호_수정'])['차대번호'].count().to_excel(os.path.join(analysis_fold, f'통계_배인번호_수정_등급_수정_{today_date}.xlsx'))

# code end